In [1]:
from preprocessing import *
import torch
import random
import json

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Find Discard Index For Training

In [2]:
try:
    f_train = open("data/discard_index.json", 'r')
    discard = json.load(f_train)['discard_index']
except:
    discard = get_discard(0.20250)

In [4]:
X_author, y = for_train('coauthor', discard)
X_author.shape

authors: 100%|██████████| 25793/25793 [00:03<00:00, 6706.67it/s]


torch.Size([9354, 21146])

In [3]:
X_sentence = for_train('word', discard, no_label=True)
X_sentence.shape

title & abstract: 100%|██████████| 25793/25793 [03:31<00:00, 121.75it/s]


torch.Size([9354, 256])

In [3]:
X_year_venue = for_train('year_venue', discard, no_label=True)
X_year_venue.shape

year: 100%|██████████| 25793/25793 [00:03<00:00, 8359.93it/s]


torch.Size([9354, 52])

#### Split the train data index and test index and save the indexes to make sure each model have the same train set

In [4]:
try:
    f_train = open("data/train_index.json", 'r')
    train_index = json.load(f_train)['train_index']

    f_test = open("data/test_index.json", 'r')
    test_index = json.load(f_test)['test_index']
except:
    
    test_size = 0.33

    test_index = random.sample(range(y.shape[0]), int(test_size * y.shape[0]))
    train_index = [i for i in range(y.shape[0]) if i not in test_index]

    train_dict = {"train_index": train_index}
    with open('data/train_index.json', 'w') as fp:
            json.dump(train_dict, fp)

    test_dict = {"test_index": test_index}
    with open('data/test_index.json', 'w') as fp:
            json.dump(test_dict, fp)

## Save the data

In [5]:
torch.save(X_sentence[train_index, :], 'data/X_sentence_train.pt')
torch.save(X_sentence[test_index, :], 'data/X_sentence_test.pt')

torch.save(X_author[train_index, :], 'data/X_author_train.pt')
torch.save(X_author[test_index, :], 'data/X_author_test.pt')

torch.save(X_year_venue[train_index, :], 'data/X_year_venue_train.pt')
torch.save(X_year_venue[test_index, :], 'data/X_year_venue_test.pt')

torch.save(y[train_index, :], 'data/y_train.pt')
torch.save(y[test_index, :], 'data/y_test.pt')